In [4]:
import pandas as pd
import requests
import os
from datetime import datetime

<h1>Téléchargement des données par saison de foot</h1>

In [2]:
# Working directory
path = os.getcwd()
path_dataset = os.path.join(path,"Datasets")

In [3]:
# Top 5 Leagues
leagues = {"E0":"England", 
           "I1":"Italy", 
           "D1":"Germany", 
           "SP1":"Spain", 
           "F1":"France"}

In [7]:
def season_list(number_season=1):
    seasons = []
    LIMIT_SAISON = 2005
    year = datetime.today().year
    # limit the max old season to 2005-2006
    if (year - number_season) < LIMIT_SAISON:
        number_season = year - LIMIT_SAISON
    #Generate season ex: ["0809",..,"1819","1920","2021",...]
    for i in range(number_season):
        year_before = year-1
        season = str(year_before)[2:] + str(year)[2:]
        seasons.append(season)
        year -= 1
    return seasons

In [11]:
# URL source dataset
url = "https://www.football-data.co.uk/mmz4281"

In [44]:
def download_datasets(league, season):
    season = str(season)
    pathdir = os.path.join(path_dataset, leagues[league])
    # Create Dataset directory if not exist
    if os.path.isdir(pathdir) == False:
        os.makedirs(pathdir)

    file_url = league+"csv"
    full_url = os.path.join(url,season,file_url)
    response = requests.get(full_url)
    file_data = os.path.join(pathdir, season+".csv")
    open(file_data, "wb").write(response.content)

In [145]:
# Download Dataset of 10 last season by top 5 leagues
seasons = season_list(10)
n_season = len(seasons)
print("Download of {} last season by League".format(n_season))

for league in leagues:
    name = leagues[league]
    n_iter = 1
    for season in seasons:
        log = "Season downloaded: {}/{} for {} League".format(n_iter, n_season, name)
        download_datasets(league, season)
        print(log,end="\r")
        n_iter += 1
    print("")

Download of 10 last season by League
Season downloaded: 10/10 for England League
Season downloaded: 10/10 for Italy League
Season downloaded: 10/10 for Germany League
Season downloaded: 10/10 for Spain League
Season downloaded: 10/10 for France League


<h1>Traitement et sélection des colonnes exploitées</h1>

In [146]:
def float2int(df):
    df = df.dropna(axis=0)
    for index, typeof in df.dtypes.items():
        if typeof == "float64":
            df[index] = df[index].astype(int)
    return df

In [147]:
# Selected columns
cols = ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'HTR', 
        'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR']

In [148]:
for dirs, _, files in os.walk(path_dataset):
    for f in files:
        path_file = dirs.split('/')[-1]
        outfile = os.path.join(dirs, path_file+"-"+f)
        file = os.path.join(dirs,f)
        df = pd.read_csv(file)
        try:
            df = df[cols]
            df = float2int(df)
        except:
            print(file)
        s = f.split('.')
        df['Season'] = str(s[0][:2])+"-"+str(s[0][2:])
        df.to_csv(outfile, index=None)
        os.remove(os.path.join(dirs,f))

/tmp/ipykernel_201296/268057422.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[index] = df[index].astype(int)
/tmp/ipykernel_201296/1829971427.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Season'] = str(s[0][:2])+"-"+str(s[0][2:])
/tmp/ipykernel_201296/268057422.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc